**Корректность проверена на Python 3.6:**
+ pandas 0.23.4
+ numpy 1.15.4
+ matplotlib 3.0.2
+ sklearn 0.20.2

# Рецензии на imdb

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
import warnings
warnings.filterwarnings('ignore')

Имеются 25000 рецензий пользователей imdb с бинарными метками, посчитанными по оценкам: 0 при оценке < 5 и 1 при оценке >=7.

Полные данные: https://www.kaggle.com/c/word2vec-nlp-tutorial/data

Загрузим выборку:

In [3]:
imdb = pd.read_csv('labeledTrainData.tsv', delimiter='\t')
imdb.shape

(25000, 3)

In [4]:
imdb.head()

,id,sentiment,review
0,5814_8,1,With all this stuff going down at the moment w...
1,2381_9,1,"\The Classic War of the Worlds\"" by Timothy Hi..."
2,7759_3,0,The film starts with a manager (Nicholas Bell)...
3,3630_4,0,It must be assumed that those who praised this...
4,9495_8,1,Superbly trashy and wondrously unpretentious 8...


Классы сбалансированы:

In [5]:
imdb.sentiment.value_counts()

1    12500
0    12500
Name: sentiment, dtype: int64

Разобъём выборку на обучение и контроль:

In [6]:
from sklearn.model_selection import train_test_split
texts_train, texts_test, y_train, y_test = train_test_split(imdb.review.values, imdb.sentiment.values)

Векторизуем тексты рецензий:

In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer
vect = TfidfVectorizer(sublinear_tf=True, use_idf=True)
X_train = vect.fit_transform(texts_train)
X_test = vect.transform(texts_test)

## Логистическая регрессия

Настроим на векторизованных данных логистическую регрессию и посчитаем AUC:

In [8]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
clf = LogisticRegression()
clf.fit(X_train, y_train)
print(metrics.accuracy_score(y_test, clf.predict(X_test)))
print(metrics.roc_auc_score(y_test, clf.predict_proba(X_test)[:, 1]))

0.89232
0.9590837335194587


Признаков получилось очень много:

In [9]:
X_train.shape

(18750, 66479)

Попробуем отбирать признаки с помощью лассо:

In [9]:
clf = LogisticRegression(C=0.15, penalty='l1')
clf.fit(X_train, y_train)
print(np.sum(np.abs(clf.coef_) > 1e-4))
print(metrics.accuracy_score(y_test, clf.predict(X_test)))
print(metrics.roc_auc_score(y_test, clf.predict_proba(X_test)[:, 1]))

112
0.81072
0.8980787993924709


Ещё один способ отбора признаков — рандомизированная логистическая регрессия:

In [10]:
from sklearn.linear_model import RandomizedLogisticRegression
rlg = RandomizedLogisticRegression(C=0.13)
rlg.fit(X_train, y_train)

ImportError: cannot import name 'RandomizedLogisticRegression' from 'sklearn.linear_model' (C:\Users\GLFS\Anaconda3\lib\site-packages\sklearn\linear_model\__init__.py)

Посмотрим, сколько признаков отбирается:

In [12]:
np.sum(rlg.scores_ > 0)

13

Настроим логистическую регрессию на отобранных признаках:

In [13]:
X_train_lasso = X_train[:, rlg.scores_ > 0]
X_test_lasso = X_test[:, rlg.scores_ > 0]

In [14]:
clf = LogisticRegression(C=1)
clf.fit(X_train_lasso, y_train)
print(metrics.accuracy_score(y_test, clf.predict(X_test_lasso)))
print(metrics.roc_auc_score(y_test, clf.predict_proba(X_test_lasso)[:, 1]))

0.73696
0.8105926603014637


### Случайный лес

In [15]:
%% time
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(n_estimators=100)
rfc.fit(X_train, y_train)
print(metrics.accuracy_score(y_test, rfc.predict(X_test)))
print(metrics.roc_auc_score(y_test, rfc.predict_proba(X_test)[:, 1]))

0.8376
0.9189803835069248


## Метод главных компонент

Сделаем 100 синтетических признаков с помощью метода главных компонент:

In [11]:
from sklearn.decomposition import TruncatedSVD
tsvd = TruncatedSVD(n_components=100)
X_train_pca = tsvd.fit_transform(X_train)
X_test_pca = tsvd.transform(X_test)

Обучим на них логистическую регрессию:

In [12]:
clf = LogisticRegression()
clf.fit(X_train_pca, y_train)
print(metrics.accuracy_score(y_test, clf.predict(X_test_pca)))
print(metrics.roc_auc_score(y_test, clf.predict_proba(X_test_pca)[:, 1]))

0.86384
0.9368912214647221


По 100 полученных таким способом признакам качество получается не намного хуже, чем по всем 66702!

Попробуем обучить на них обучить случайный лес:

In [13]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators=100)
clf.fit(X_train_pca, y_train)
print(metrics.accuracy_score(y_test, clf.predict(X_test_pca)))
print(metrics.roc_auc_score(y_test, clf.predict_proba(X_test_pca)[:, 1]))

0.828
0.9084036628704282


Признаки, которые даёт метод главных компонент, оптимальны для линейных методов, поэтому логистическая регрессия показывает результаты лучше, чем сложные нелинейные классификаторы.